In [116]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data=pd.read_excel('/Users/zhongrunping/Desktop/jupyter文件夹/数学建模/2020年B题/数模题/附件三：285号和313号样本原始数据的副本.xlsx'
              ,sheet_name=4
             )
data=data[1:].reset_index(drop=True)

In [3]:
sample285=data[:40].iloc[:,1:]
sample313=data[40:].reset_index(drop=True).iloc[:,1:]

In [4]:
#检查是否有缺失值
sample285.isnull().sum()[sample285.isnull().sum()>0]
#没有缺失值

Series([], dtype: int64)

In [5]:
#读取原始变量取值范围
interval=pd.read_excel('/Users/zhongrunping/Desktop/jupyter文件夹/数学建模/2020年B题/数模题/附件四：354个操作变量信息.xlsx')

In [6]:
interval=pd.concat([interval['位号'],interval['取值范围']],axis=1)
interval.head()

,位号,取值范围
0,S-ZORB.CAL_H2.PV,0.20-0.37
1,S-ZORB.PDI_2102.PV,10-35
2,S-ZORB.PT_2801.PV,2.35-2.70
3,S-ZORB.FC_2801.PV,600-1000
4,S-ZORB.TE_2103.PV,410-435


In [7]:
def get_interval(x):
    x=x.split('-')
    if len(x)==2:
        maximum=float(x[1])
        minmum=float(x[0])
        return minmum,maximum
    else:
        return x
interval['取值范围']=interval['取值范围'].apply(get_interval)

In [8]:
#interval.to_excel('各个变量的取值范围.xlsx',index=None)

In [10]:
new_interval=pd.read_excel('各个变量的取值范围的副本.xlsx')

In [11]:
def get_num(x):
    return x.strip('()').split(',')
new_interval['取值范围']=new_interval['取值范围'].apply(get_num)

In [12]:
maximum=[]
minimum=[]
for j,i in enumerate(new_interval['取值范围']):
    minimum.append(float(i[0]))
    maximum.append(float(i[1]))

In [13]:
new_interval['最小值']=minimum
new_interval['最大值']=maximum

In [14]:
new_interval=new_interval.drop(['取值范围'],axis=1)

In [126]:
#new_interval.to_excel('各变量的最大最小值.xlsx',index=None)

In [128]:
new_interval

,位号,最小值,最大值
0,S-ZORB.CAL_H2.PV,0.20,0.37
1,S-ZORB.PDI_2102.PV,10.00,35.00
2,S-ZORB.PT_2801.PV,2.35,2.70
3,S-ZORB.FC_2801.PV,600.00,1000.00
4,S-ZORB.TE_2103.PV,410.00,435.00
...,...,...,...
349,S-ZORB.FT_1503.DACA.PV,60.00,5500000.00
350,S-ZORB.FT_1503.TOTALIZERA.PV,0.00,40000000.00
351,S-ZORB.FT_1504.DACA.PV,-14000.00,3500.00
352,S-ZORB.FT_1504.TOTALIZERA.PV,0.00,40000000.00


In [17]:
#获取变量的取值范围
def get_column_range(column):
    index=new_interval['位号'][new_interval['位号']==column].index[0]
    maximum=new_interval['最大值'][index]
    minimum=new_interval['最小值'][index]
    return minimum,maximum  #返回最大最小值
#将超出范围的值设置为空
for column in sample285.columns:
    a,b=get_column_range(column)
    sample285[column]=sample285[column].apply(lambda x:np.nan if x<a or x>b else x)
for column in sample313.columns:
    a,b=get_column_range(column)
    sample313[column]=sample313[column].apply(lambda x:np.nan if x<a or x>b else x)

In [44]:
#将不在3sigma以内的设置为空
def three_sigma(df):
    mean=df.mean()
    std=df.std()
    df=df.apply(lambda x :np.nan if x<mean-3*std or x>mean+3*std else x)
for column in sample285.columns:
    three_sigma(sample285[column])
for column in sample313.columns:
    three_sigma(sample313[column])

In [47]:
#查看缺失值
sample285.isnull().sum()[sample285.isnull().sum()>0]

S-ZORB.SIS_LT_1001.PV    40
S-ZORB.AI_2903.PV        40
S-ZORB.FT_1202.TOTAL     40
S-ZORB.FT_1204.TOTAL     40
dtype: int64

In [48]:
sample313.isnull().sum()[sample313.isnull().sum()>0]

S-ZORB.AT_5201.PV          39
S-ZORB.PT_9403.PV           3
S-ZORB.FT_9402.PV           1
S-ZORB.PDC_2502.PV         20
S-ZORB.FC_2501.PV           4
S-ZORB.SIS_LT_1001.PV      40
S-ZORB.PC_6001.PV           2
S-ZORB.PT_6002.PV           5
S-ZORB.AI_2903.PV          40
S-ZORB.PDC_2607.PV          2
S-ZORB.FT_1202.TOTAL       40
S-ZORB.FT_1204.PV           2
S-ZORB.FT_1204.TOTAL       40
S-ZORB.PC_3101.DACA         1
S-ZORB.PT_2501.DACA         8
S-ZORB.PT_2502.DACA        10
S-ZORB.FC_2432.DACA         6
S-ZORB.FT_2431.DACA         8
S-ZORB.PDI_2801.DACA        3
S-ZORB.PDI_2301.DACA        3
S-ZORB.BS_LT_2401.PV       14
S-ZORB.PC_2401.DACA         4
S-ZORB.PC_2401B.DACA        3
S-ZORB.PC_2401B.PIDA.SP     3
S-ZORB.PC_2401B.PIDA.OP     4
S-ZORB.PC_2401.PIDA.OP      7
S-ZORB.PC_2401.PIDA.SP      6
S-ZORB.PDT_2409.DACA        2
S-ZORB.TE_1603.DACA         9
S-ZORB.AT-0006.DACA.PV      7
S-ZORB.AT-0012.DACA.PV      3
dtype: int64

In [54]:
#将313号样本缺失比例大于80%的设置为空
null=(sample313.isnull().sum()[sample313.isnull().sum()>0]/len(sample313))[sample313.isnull().sum()[sample313.isnull().sum()>0]/len(sample313)>0.8]
null

S-ZORB.AT_5201.PV        0.975
S-ZORB.SIS_LT_1001.PV    1.000
S-ZORB.AI_2903.PV        1.000
S-ZORB.FT_1202.TOTAL     1.000
S-ZORB.FT_1204.TOTAL     1.000
dtype: float64

In [55]:
for i in null.index:
    sample313[i]=np.nan

In [75]:
#填补缺失值
for i in sample313.columns:
    sample313[i]=sample313[i].fillna(sample313[i].mean())
#查看缺失值比例
sample313.isnull().sum()[sample313.isnull().sum()>0]/40

S-ZORB.AT_5201.PV        1.0
S-ZORB.SIS_LT_1001.PV    1.0
S-ZORB.AI_2903.PV        1.0
S-ZORB.FT_1202.TOTAL     1.0
S-ZORB.FT_1204.TOTAL     1.0
dtype: float64

In [76]:
sample285.isnull().sum()[sample285.isnull().sum()>0]/40

S-ZORB.SIS_LT_1001.PV    1.0
S-ZORB.AI_2903.PV        1.0
S-ZORB.FT_1202.TOTAL     1.0
S-ZORB.FT_1204.TOTAL     1.0
dtype: float64

In [88]:
result_of_285=pd.DataFrame(sample285.describe().iloc[1,:]).T
result_of_313=pd.DataFrame(sample313.describe().iloc[1,:]).T

In [92]:
result_of_285.isnull().sum()[result_of_285.isnull().sum()>0]

S-ZORB.SIS_LT_1001.PV    1
S-ZORB.AI_2903.PV        1
S-ZORB.FT_1202.TOTAL     1
S-ZORB.FT_1204.TOTAL     1
dtype: int64

In [93]:
result_of_313.isnull().sum()[result_of_313.isnull().sum()>0]

S-ZORB.AT_5201.PV        1
S-ZORB.SIS_LT_1001.PV    1
S-ZORB.AI_2903.PV        1
S-ZORB.FT_1202.TOTAL     1
S-ZORB.FT_1204.TOTAL     1
dtype: int64

In [111]:
fina=pd.concat([result_of_285,result_of_313])
fina.index=[285,313]

In [122]:
fina.to_excel('第一问结果.xlsx',na_rep='Nan')

In [119]:
fina.isna().sum()[fina.isna().sum()>0]

S-ZORB.AT_5201.PV        1
S-ZORB.SIS_LT_1001.PV    2
S-ZORB.AI_2903.PV        2
S-ZORB.FT_1202.TOTAL     2
S-ZORB.FT_1204.TOTAL     2
dtype: int64

In [127]:
fina

,S-ZORB.CAL_H2.PV,S-ZORB.PDI_2102.PV,S-ZORB.PT_2801.PV,S-ZORB.FC_2801.PV,S-ZORB.TE_2103.PV,S-ZORB.TE_2005.PV,S-ZORB.PT_2101.PV,S-ZORB.PDT_2104.PV,S-ZORB.SIS_PDT_2103B.PV,S-ZORB.TC_2101.PV,...,S-ZORB.CAL_1.CANGLIANG.PV,S-ZORB.FT_1006.DACA.PV,S-ZORB.FT_5204.DACA.PV,S-ZORB.FT_1006.TOTALIZERA.PV,S-ZORB.FT_5204.TOTALIZERA.PV,S-ZORB.FT_1503.DACA.PV,S-ZORB.FT_1503.TOTALIZERA.PV,S-ZORB.FT_1504.DACA.PV,S-ZORB.FT_1504.TOTALIZERA.PV,S-ZORB.PC_1001A.PV
285,0.273399,24.208241,2.528870,855.882523,421.509325,421.196235,2.427093,59.703011,1108.285375,244.121748,...,2.358559,3321.583200,190.694200,98944916.75,2433448.000,2200.789100,5149259.0,2846.896600,5984749.325,-97.210697
313,0.261936,17.183467,2.417056,850.300855,424.968595,424.522167,2.315898,61.701366,1250.692900,245.333915,...,2.894837,10013.864775,147.165737,87939278.25,2205865.275,1943.691157,2149491.8,2501.854175,2154163.850,-113.375918
